## Libraries

In [7]:
import qiime2
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition
import pandas as pd
import os

## Functions

## Main

In [24]:
df = pd.read_csv("./data/amerinca_gut.txt", delimiter="\t", header=0)
df.columns = df.columns.str.replace(' ', '_')

In [32]:
df.shape

(29960, 661)

In [33]:
df.columns

Index(['sample_name', 'acid_reflux', 'acne_medication', 'acne_medication_otc',
       'add_adhd', 'age_cat', 'age_corrected', 'age_years',
       'alcohol_consumption', 'alcohol_frequency',
       ...
       'vioscreen_zinc', 'vitamin_b_supplement_frequency',
       'vitamin_d_supplement_frequency', 'vivid_dreams', 'weight_cat',
       'weight_change', 'weight_kg', 'weight_units', 'whole_eggs',
       'whole_grain_frequency'],
      dtype='object', length=661)

In [43]:
df['alcohol_frequency'].value_counts()

Rarely (a few times/month)       6106
Never                            5992
Occasionally (1-2 times/week)    5528
Regularly (3-5 times/week)       4759
Not applicable                   3598
Daily                            2488
LabControl test                   681
Not provided                      437
Unspecified                       281
not applicable                     10
not collected                       3
Name: alcohol_frequency, dtype: int64

In [44]:
df['sample_name']

0                10317.000001000
1                10317.000001001
2                10317.000001002
3                10317.000001004
4                10317.000001008
                  ...           
29955          10317.blank275.2F
29956          10317.blank275.2G
29957          10317.blank275.2H
29958    10317.blank3.ragOffice1
29959    10317.blank4.ragOffice1
Name: sample_name, Length: 29960, dtype: object